In [20]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
import json
import datetime 

#access the dataset and extract Copd, cardiovascular disease dataset
#Generate basic statistics 
#ROW_ID is just numberof rows, SUBJECT_ID refers to a unique patient
#Find ID to match disease to patient


In [70]:
#df = pd.read_fwf('/data/jb2297/physionet.org/files/mimiciii/1.4/README.txt')
#index = pd.read_html('/data/jb2297/physionet.org/files/mimiciii/1.4/index.html')

In [66]:
data = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/DIAGNOSES_ICD.csv')
#data.head()
print(data)
print(data.describe())
#how to match diagnosis to disease?

        ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
0         1297         109   172335      1.0     40301
1         1298         109   172335      2.0       486
2         1299         109   172335      3.0     58281
3         1300         109   172335      4.0      5855
4         1301         109   172335      5.0      4254
...        ...         ...      ...      ...       ...
651042  639798       97503   188195      2.0     20280
651043  639799       97503   188195      3.0     V5869
651044  639800       97503   188195      4.0     V1279
651045  639801       97503   188195      5.0      5275
651046  639802       97503   188195      6.0      5569

[651047 rows x 5 columns]
              ROW_ID     SUBJECT_ID        HADM_ID        SEQ_NUM
count  651047.000000  651047.000000  651047.000000  651000.000000
mean   325524.000000   38971.159758  150017.744819       7.913836
std    187941.224689   29372.198841   28878.068648       6.072633
min         1.000000       2.000000  100001.00000

In [67]:
patient = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/PATIENTS.csv')
print(patient)
print(patient.describe())


       ROW_ID  SUBJECT_ID GENDER                  DOB                  DOD  \
0         234         249      F  2075-03-13 00:00:00                  NaN   
1         235         250      F  2164-12-27 00:00:00  2188-11-22 00:00:00   
2         236         251      M  2090-03-15 00:00:00                  NaN   
3         237         252      M  2078-03-06 00:00:00                  NaN   
4         238         253      F  2089-11-26 00:00:00                  NaN   
...       ...         ...    ...                  ...                  ...   
46515   31840       44089      M  2026-05-25 00:00:00                  NaN   
46516   31841       44115      F  2124-07-27 00:00:00                  NaN   
46517   31842       44123      F  2049-11-26 00:00:00  2135-01-12 00:00:00   
46518   31843       44126      F  2076-07-25 00:00:00                  NaN   
46519   31844       44128      M  2098-07-25 00:00:00                  NaN   

                  DOD_HOSP DOD_SSN  EXPIRE_FLAG  
0            

In [23]:
data3 = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/D_ICD_DIAGNOSES.csv')
print(data3)

       ROW_ID ICD9_CODE               SHORT_TITLE  \
0         174     01166     TB pneumonia-oth test   
1         175     01170    TB pneumothorax-unspec   
2         176     01171   TB pneumothorax-no exam   
3         177     01172  TB pneumothorx-exam unkn   
4         178     01173  TB pneumothorax-micro dx   
...       ...       ...                       ...   
14562   14432     V7399     Scrn unspcf viral dis   
14563   14433      V740     Screening for cholera   
14564   14434      V741    Screening-pulmonary TB   
14565   14435      V742     Screening for leprosy   
14566   14436      V743  Screening for diphtheria   

                                              LONG_TITLE  
0      Tuberculous pneumonia [any form], tubercle bac...  
1                  Tuberculous pneumothorax, unspecified  
2      Tuberculous pneumothorax, bacteriological or h...  
3      Tuberculous pneumothorax, bacteriological or h...  
4      Tuberculous pneumothorax, tubercle bacilli fou...  
...      

In [77]:
#find disease name of rowid = 1297
#finddata = data3.loc[data['ROW_ID']==1297]
#findcopd=data3.loc[data['LONG_TITLE ']=='Chronic obstructive pulmonary disease']
#print(findcopd)

#findcardio=data3.loc[data3['LONG_TITLE']=='Chronic obstructive pulmonary disease']
#print(findcardio)

findtb=data3[data3['SHORT_TITLE'].str.contains('TB')]
#print(findtb)

findcardio=data3[data3['SHORT_TITLE'].str.contains('Cardiovascular')]
#print(findcardio)

findcardio2=data3[data3['LONG_TITLE'].str.contains('Cardiovascular')]
#print(findcardio2)

findcardio3=data3[data3['SHORT_TITLE'].str.contains('Cardio|cardio|heart|Heart|obstructive pulmonary')]
print(findcardio3)

# include long title?




       ROW_ID ICD9_CODE               SHORT_TITLE  \
318       369      0860   Chagas disease of heart   
372       423     09389   Cardiovascular syph NEC   
373       424      0939   Cardiovascular syph NOS   
437       488     09885  Gonococcal heart dis NEC   
951      1332      1641     Malignant neopl heart   
...       ...       ...                       ...   
13991   11613     E8583  Acc poisn-cardiovasc agt   
14008   11630     99671   Comp-heart valve prosth   
14025   13402     E8745  Instrmnt fail-heart cath   
14191   14061      V717   Obs-susp cardiovasc dis   
14214   14084     V7281    Preop cardiovsclr exam   

                                              LONG_TITLE  
318               Chagas' disease with heart involvement  
372              Other specified cardiovascular syphilis  
373                 Cardiovascular syphilis, unspecified  
437                       Other gonococcal heart disease  
951                          Malignant neoplasm of heart  
...      

In [78]:
findcardio3.describe()
#count number of patients with a disease. In different csv?

,ROW_ID
count,89.000000
mean,8543.292135
std,3977.845112
min,369.000000
25%,4439.000000
50%,8592.000000
75%,12285.000000
max,14503.000000


In [83]:
#DIAGNOSES_ICD.csv same as D_ICD_DIAGNOSES.csv
DICDdiagnosesdata = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/D_ICD_DIAGNOSES.csv')
#print(DICDdiagnosesdata)

       ROW_ID ICD9_CODE               SHORT_TITLE  \
0         174     01166     TB pneumonia-oth test   
1         175     01170    TB pneumothorax-unspec   
2         176     01171   TB pneumothorax-no exam   
3         177     01172  TB pneumothorx-exam unkn   
4         178     01173  TB pneumothorax-micro dx   
...       ...       ...                       ...   
14562   14432     V7399     Scrn unspcf viral dis   
14563   14433      V740     Screening for cholera   
14564   14434      V741    Screening-pulmonary TB   
14565   14435      V742     Screening for leprosy   
14566   14436      V743  Screening for diphtheria   

                                              LONG_TITLE  
0      Tuberculous pneumonia [any form], tubercle bac...  
1                  Tuberculous pneumothorax, unspecified  
2      Tuberculous pneumothorax, bacteriological or h...  
3      Tuberculous pneumothorax, bacteriological or h...  
4      Tuberculous pneumothorax, tubercle bacilli fou...  
...      

In [65]:
prescriptdata = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/PRESCRIPTIONS.csv')
print(prescriptdata)


/data/jb2297/Anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


          ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID            STARTDATE  \
0        2214776           6   107064         NaN  2175-06-11 00:00:00   
1        2214775           6   107064         NaN  2175-06-11 00:00:00   
2        2215524           6   107064         NaN  2175-06-11 00:00:00   
3        2216265           6   107064         NaN  2175-06-11 00:00:00   
4        2214773           6   107064         NaN  2175-06-11 00:00:00   
...          ...         ...      ...         ...                  ...   
4156445  3996662       98887   121032    238144.0  2144-09-06 00:00:00   
4156446  3996070       98887   121032    238144.0  2144-09-06 00:00:00   
4156447  3996063       98887   121032    238144.0  2144-09-06 00:00:00   
4156448  3996062       98887   121032    238144.0  2144-09-06 00:00:00   
4156449  3996061       98887   121032    238144.0  2144-09-06 00:00:00   

                     ENDDATE DRUG_TYPE                      DRUG  \
0        2175-06-12 00:00:00      MAIN     

In [29]:
admissiondata = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/ADMISSIONS.csv')
print(admissiondata)



       ROW_ID  SUBJECT_ID  HADM_ID            ADMITTIME            DISCHTIME  \
0          21          22   165315  2196-04-09 12:26:00  2196-04-10 15:54:00   
1          22          23   152223  2153-09-03 07:15:00  2153-09-08 19:10:00   
2          23          23   124321  2157-10-18 19:34:00  2157-10-25 14:00:00   
3          24          24   161859  2139-06-06 16:14:00  2139-06-09 12:48:00   
4          25          25   129635  2160-11-02 02:06:00  2160-11-05 14:55:00   
...       ...         ...      ...                  ...                  ...   
58971   58594       98800   191113  2131-03-30 21:13:00  2131-04-02 15:02:00   
58972   58595       98802   101071  2151-03-05 20:00:00  2151-03-06 09:10:00   
58973   58596       98805   122631  2200-09-12 07:15:00  2200-09-20 12:08:00   
58974   58597       98813   170407  2128-11-11 02:29:00  2128-12-22 13:11:00   
58975   58598       98813   190264  2131-10-25 03:09:00  2131-10-26 17:44:00   

                 DEATHTIME ADMISSION_TY

In [30]:
ditemdata = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/D_ITEMS.csv')
print(ditemdata)


       ROW_ID  ITEMID                                        LABEL  \
0         457     497  Patient controlled analgesia (PCA) [Inject]   
1         458     498                            PCA Lockout (Min)   
2         459     499                               PCA Medication   
3         460     500                               PCA Total Dose   
4         461     501                           PCV Exh Vt (Obser)   
...       ...     ...                                          ...   
12482   14518  226757                        GCSMotorApacheIIValue   
12483   14519  226758                       GCSVerbalApacheIIValue   
12484   14520  226759                            HCO3ApacheIIValue   
12485   14521  226760                                    HCO3Score   
12486   14522  226761                      HematocritApacheIIScore   

                  ABBREVIATION    DBSOURCE      LINKSTO            CATEGORY  \
0                          NaN     carevue  chartevents                 NaN   
1

In [64]:
noteventdata = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/NOTEEVENTS.csv')
print(noteventdata)

/data/jb2297/Anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


          ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE            CHARTTIME  \
0            174       22532  167853.0  2151-08-04                  NaN   
1            175       13702  107527.0  2118-06-14                  NaN   
2            176       13702  167118.0  2119-05-25                  NaN   
3            177       13702  196489.0  2124-08-18                  NaN   
4            178       26880  135453.0  2162-03-25                  NaN   
...          ...         ...       ...         ...                  ...   
2083175  2070657       31097  115637.0  2132-01-21  2132-01-21 03:27:00   
2083176  2070658       31097  115637.0  2132-01-21  2132-01-21 09:50:00   
2083177  2070659       31097  115637.0  2132-01-21  2132-01-21 16:42:00   
2083178  2070660       31097  115637.0  2132-01-21  2132-01-21 18:05:00   
2083179  2070661       31097  115637.0  2132-01-21  2132-01-21 18:05:00   

                   STORETIME           CATEGORY DESCRIPTION     CGID  ISERROR  \
0                 

In [79]:
calldata = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/CALLOUT.csv')
print(calldata)

       ROW_ID  SUBJECT_ID  HADM_ID  SUBMIT_WARDID SUBMIT_CAREUNIT  \
0         402         854   175684           52.0             NaN   
1         403         864   138624           15.0             NaN   
2         404         864   138624           12.0             NaN   
3         405         867   184298            7.0             NaN   
4         157         306   167129           57.0             NaN   
...       ...         ...      ...            ...             ...   
34494   32674       94046   199742           23.0             NaN   
34495   32675       94049   135809           23.0             NaN   
34496   32676       94050   173013           23.0             NaN   
34497   32677       94056   172374           50.0             NaN   
34498   32678       94057   113695           23.0            MICU   

       CURR_WARDID CURR_CAREUNIT  CALLOUT_WARDID CALLOUT_SERVICE  \
0             29.0          MICU               1             MED   
1             55.0          CSRU   

In [80]:
dlabdata = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/D_LABITEMS.csv')
print(dlabdata)



     ROW_ID  ITEMID                       LABEL                      FLUID  \
0       546   51346                      Blasts  Cerebrospinal Fluid (CSF)   
1       547   51347                 Eosinophils  Cerebrospinal Fluid (CSF)   
2       548   51348             Hematocrit, CSF  Cerebrospinal Fluid (CSF)   
3       549   51349  Hypersegmented Neutrophils  Cerebrospinal Fluid (CSF)   
4       550   51350           Immunophenotyping  Cerebrospinal Fluid (CSF)   
..      ...     ...                         ...                        ...   
748     749   51551             VOIDED SPECIMEN           OTHER BODY FLUID   
749     750   51552             VOIDED SPECIMEN                      STOOL   
750     751   51553             VOIDED SPECIMEN                      URINE   
751     752   51554             VOIDED SPECIMEN                JOINT FLUID   
752     753   51555    SURFACTANT ALBUMIN RATIO           OTHER BODY FLUID   

       CATEGORY LOINC_CODE  
0    Hematology    26447-3  
1    

In [81]:
outputdata = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/OUTPUTEVENTS.csv')
print(outputdata)

          ROW_ID  SUBJECT_ID   HADM_ID  ICUSTAY_ID            CHARTTIME  \
0            344       21219  177991.0    225765.0  2142-09-08 10:00:00   
1            345       21219  177991.0    225765.0  2142-09-08 12:00:00   
2            346       21219  177991.0    225765.0  2142-09-08 13:00:00   
3            347       21219  177991.0    225765.0  2142-09-08 14:00:00   
4            348       21219  177991.0    225765.0  2142-09-08 16:00:00   
...          ...         ...       ...         ...                  ...   
4349213  4340476       68375  123645.0    276251.0  2175-09-05 23:00:00   
4349214  4340477       68375  123645.0    276251.0  2175-09-06 00:00:00   
4349215  4340478       68375  123645.0    276251.0  2175-09-06 01:00:00   
4349216  4340479       68375  123645.0    276251.0  2175-09-06 02:00:00   
4349217  4340480       68375  123645.0    276251.0  2175-09-06 04:00:00   

         ITEMID  VALUE VALUEUOM            STORETIME   CGID  STOPPED  \
0         40055  200.0     

In [82]:
#CAREGIVERS
icudata = pd.read_csv('/data/jb2297/physionet.org/files/mimiciii/1.4/ICUSTAYS.csv')
print(icudata)

       ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID    DBSOURCE FIRST_CAREUNIT  \
0         365         268   110404      280836     carevue           MICU   
1         366         269   106296      206613     carevue           MICU   
2         367         270   188028      220345     carevue            CCU   
3         368         271   173727      249196     carevue           MICU   
4         369         272   164716      210407     carevue            CCU   
...       ...         ...      ...         ...         ...            ...   
61527   59806       94944   143774      201233  metavision           CSRU   
61528   59807       94950   123750      283653  metavision            CCU   
61529   59808       94953   196881      241585  metavision           SICU   
61530   59809       94954   118475      202802  metavision           CSRU   
61531   59810       94956   156386      275346  metavision          TSICU   

      LAST_CAREUNIT  FIRST_WARDID  LAST_WARDID               INTIME  \
0   